In [2]:
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

In [3]:
#Importo
df_producto=pd.read_csv('../data/EXPO_DOC.csv', encoding='latin-1',sep=';', dtype={'CUIT':'str',
                                                                                            'dia':'str',
                                                                                            'mes':'str',
                                                                                            'anio':'str'})
# Correcciones de lectura
df_producto.pais_descri=df_producto.pais_descri.apply(lambda x: x.capitalize())
df_producto.empresa=df_producto.empresa.apply(lambda x: x.lower())

#Fob por tonelada importada
df_producto['fob_unitario_ton']=df_producto.fob/df_producto.pnet*1000

#Importo datos de los precios de referencia del banco mundial
bm=pd.read_excel('../data/CMO-Historical-Data-Monthly.xlsx',sheet_name='Monthly Prices',skiprows=6)
bm=bm[[bm.columns[0],bm.columns[24],bm.columns[30],bm.columns[37]]].rename(columns={'Unnamed: 0':'Fecha', 'SOYBEANS':'soja_bm',bm.columns[37]:'trigo_bm',bm.columns[30]:'maiz_bm' })
bm['anio']=bm.Fecha.apply(lambda x: x[:4])
bm['mes']=bm.Fecha.apply(lambda x: x[5:])
bm.drop('Fecha',axis=1,inplace=True)

# Separo las dataframes para cada producto, haciendolo mas facil de manipular
dfs_producto=[i for x, i in df_producto.groupby('NOMEN', as_index=False)]
trigo=pd.merge(left=dfs_producto[0],right=bm[['anio','mes','trigo_bm']], on=['anio','mes'],how='left').rename({'trigo_bm':'bm'},axis=1) #concat
trigo['fob_unitario_ton_capit']=trigo.fob_unitario_ton*trigo.bm.iloc[-1]/trigo.bm
trigo['diferencia_ref']=trigo.fob_unitario_ton-trigo.bm
trigo['diferencia_ref_capi']=trigo.fob_unitario_ton_capit-trigo.bm.iloc[-1]
trigo['fecha']=trigo["mes"].astype(str)+'-'+trigo['anio'].astype(str)
trigo['fecha']=pd.to_datetime(trigo["fecha"]).dt.strftime('%m-%Y')
trigo=trigo.sort_values(['anio','mes','dia'], ascending=True)

maiz=pd.merge(left=dfs_producto[1],right=bm[['anio','mes','maiz_bm']], on=['anio','mes'],how='left').rename({'maiz_bm':'bm'},axis=1) #concat
maiz['fob_unitario_ton_capit']=maiz.fob_unitario_ton*maiz.bm.iloc[-1]/maiz.bm
maiz['diferencia_ref']=maiz.fob_unitario_ton-maiz.bm
maiz['diferencia_ref_capi']=maiz.fob_unitario_ton_capit-maiz.bm.iloc[-1]
maiz['fecha']=maiz["mes"].astype(str)+'-'+maiz['anio'].astype(str)
maiz['fecha']=pd.to_datetime(maiz["fecha"]).dt.strftime('%m-%Y')
maiz=maiz.sort_values(['anio','mes','dia'], ascending=True)

soja=pd.merge(left=dfs_producto[2],right=bm[['anio','mes','soja_bm']], on=['anio','mes'],how='left').rename({'soja_bm':'bm'},axis=1) #concat
soja['fob_unitario_ton_capit']=soja.fob_unitario_ton*soja.bm.iloc[-1]/soja.bm
soja['diferencia_ref']=soja.fob_unitario_ton-soja.bm
soja['diferencia_ref_capi']=soja.fob_unitario_ton_capit-soja.bm.iloc[-1]
soja['fecha']=soja["mes"].astype(str)+'-'+soja['anio'].astype(str)
soja['fecha']=pd.to_datetime(soja["fecha"]).dt.strftime('%m-%Y')
soja=soja.sort_values(['anio','mes','dia'], ascending=True)


desde=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'][0]
hasta=df_producto.sort_values(['anio','mes','dia'], ascending=True)['anio'].iloc[-1]

In [4]:
trigo_outliers=trigo.sort_values('fob_unitario_ton',ascending=False)
trigo_outliers=trigo_outliers[['anio','dia','mes','NOMEN','CUIT','empresa','pais_descri','fob','pnet','fob_unitario_ton','bm']]

# Outliers a partir de 1000 en adelante. 
trigo=trigo[trigo.fob_unitario_ton<1000].reset_index(drop=True)

trigo_outliers.head(15) #Para ver outliers

,anio,dia,mes,NOMEN,CUIT,empresa,pais_descri,fob,pnet,fob_unitario_ton,bm
1049,2022,21,04,10019900,33714472319,biotrigo genetica srl,Brasil,52.80,52.80,1000.000000,495.28
1081,2022,22,01,10019900,33714472319,biotrigo genetica srl,Brasil,863.33,863.33,1000.000000,374.24
992,2021,20,03,10019900,33714472319,biotrigo genetica srl,Brasil,728.00,728.00,1000.000000,273.13
295,2021,06,05,10019900,33714472319,biotrigo genetica srl,Brasil,98.15,98.15,1000.000000,297.25
1424,2022,28,02,10019900,33714472319,biotrigo genetica srl,Brasil,1006.15,1006.15,1000.000000,390.50
1447,2022,28,07,10019900,30712354530,edco grains s.a,Brasil,14560.00,28000.00,520.000000,382.50
447,2022,09,06,10019900,30712354530,edco grains s.a,Brasil,14000.00,28000.00,500.000000,459.59
1444,2022,28,06,10019900,30712354530,edco grains s.a,Brasil,28000.00,56000.00,500.000000,459.59
353,2022,07,06,10019900,30541934029,molino matilde s a,Brasil,200011.74,401630.00,498.000000,459.59
1496,2022,29,06,10019900,30709689726,jewell especialidades sa,Ecuador,75325.32,151560.00,497.000000,459.59


In [5]:
maiz_outliers=maiz.sort_values('fob_unitario_ton',ascending=False)
maiz_outliers=maiz_outliers[['anio','dia','mes','NOMEN','CUIT','empresa','pais_descri','fob','pnet','fob_unitario_ton','bm']]

# Outliers a partir de 1000 en adelante. 
maiz=maiz[maiz.fob_unitario_ton<1000].reset_index(drop=True)

maiz_outliers.head(15)

,anio,dia,mes,NOMEN,CUIT,empresa,pais_descri,fob,pnet,fob_unitario_ton,bm
4674,2022,22,02,10059010,30708792493,snack crops sociedad anonima,Italia,16670.99,26.38,631955.648218,292.622344
3762,2021,17,10,10059010,30710254830,dos mates sa,Libano,29269.04,51.53,568000.000000,239.648763
5869,2022,27,01,10059010,30716887614,beans export s.r.l.,Uruguay,22575.00,150.50,150000.000000,276.623189
371,2021,02,10,10059010,30715780026,powertec comercio s.r.l,Uruguay,28817.60,205.84,140000.000000,239.648763
5267,2021,24,08,10059010,27220974761,rojas luisa beatriz,Bolivia,34800.00,290.00,120000.000000,256.612434
4604,2021,21,10,10059010,30541588619,molino passerini saic,Chile,840.00,600.00,1400.000000,239.648763
5026,2021,23,08,10059010,30541588619,molino passerini saic,Chile,134.00,100.00,1340.000000,256.612434
3176,2021,15,04,10059010,30541588619,molino passerini saic,Chile,310.00,250.00,1240.000000,268.233868
1336,2022,07,01,10059010,30714682055,keymar srl,Estados unidos,2534.40,2087.42,1214.130362,276.623189
4722,2021,22,04,10059010,30708499818,seedar s.a.,Brasil,1242.50,1250.00,994.000000,268.233868


In [6]:
soja_outliers=soja.sort_values('fob_unitario_ton',ascending=False)
soja_outliers=soja_outliers[['anio','dia','mes','NOMEN','CUIT','empresa','pais_descri','fob','pnet','fob_unitario_ton','bm']]
# Outliers a partir de 1000 en adelante. 
soja=soja[soja.fob_unitario_ton<3000].reset_index(drop=True)

soja_outliers.head(15)

,anio,dia,mes,NOMEN,CUIT,empresa,pais_descri,fob,pnet,fob_unitario_ton,bm
464,2022,14,02,12019000,30716309122,trigus trading s.a.,Uruguay,61954.40,144.08,430000.000000,661.63
327,2022,10,02,12019000,30716309122,trigus trading s.a.,Uruguay,61799.60,143.72,430000.000000,661.63
440,2022,13,04,12019000,30716891182,jm y cia s.a.s.,Chile,76500.00,17000.00,4500.000000,720.79
194,2022,06,07,12019000,30630720911,petroagro sociedad anonima,Estados unidos,142642.50,150150.00,950.000000,678.20
479,2022,14,07,12019000,30630720911,petroagro sociedad anonima,Estados unidos,7298983.00,7683140.00,950.000000,678.20
553,2022,16,07,12019000,30630720911,petroagro sociedad anonima,Estados unidos,244530.00,257400.00,950.000000,678.20
762,2022,23,06,12019000,30630720911,petroagro sociedad anonima,Estados unidos,224276.00,236080.00,950.000000,737.06
934,2022,28,07,12019000,30630720911,petroagro sociedad anonima,Estados unidos,244311.50,257170.00,950.000000,678.20
804,2021,24,07,12019000,30710674570,roagro negocios s. a.,Canada,190048.10,203260.00,935.000000,600.44
897,2021,27,06,12019000,30710674570,roagro negocios s. a.,Canada,103710.20,110920.00,935.000000,614.68


In [7]:
def plot_precio(y, color, producto):
     '''y='fob_unitario_ton', 'diferencia_ref'
     color='pais_descri','empresa'     
     Producto='trigo','soja','maiz'''
     if producto=='soja':
          df=soja
     elif producto=='trigo':
          df=trigo
     elif producto=='maiz':
          df=maiz
     
     if y=='fob_unitario_ton':
          title_1= 'Fob unitario'
          title_2= f'FOB por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial'
     elif y=='diferencia_ref':
          title_1= 'Diferencia en USD'
          title_2= f'Diferencia con precio de referencia en USD por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial'
     precio_soja_plot=px.scatter(
     df.round(1),
     x='fecha',
     y=y,
     color=color,
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_plot.update_yaxes(title_text= title_1, 
                                   range=[df[y].min()-100,df[y].max()+100]
                                   )
     precio_soja_plot.update_xaxes(title_text='')

     precio_soja_plot.update_traces(marker=dict(size=12,
                                   line=dict(width=2,
                                             color='DarkSlateGrey')),
                    selector=dict(mode='markers'))

     precio_soja_plot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=900,
                                   template = 'none',
                                   title_text=title_2)

     return precio_soja_plot

def precio_boxplot_mes(producto):
     '''Producto='trigo','soja','maiz'''
     if producto=='soja':
          df=soja
     elif producto=='trigo':
          df=trigo
     elif producto=='maiz':
          df=maiz
     
     precio_soja_boxplot=px.box(
     df.round(1),
     x='fecha',
     y='fob_unitario_ton',
     color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= 'Fob unitario', 
                                   range=[df['fob_unitario_ton'].min()-100,df['fob_unitario_ton'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=700, width=900,
                                   template = 'none',
                                   title_text=f'FOB por tonelada exportada de {producto} <br> <sup> Precio de referencia del Banco Mundial',
                                   showlegend=False)

     return precio_soja_boxplot

def precio_boxplot_capitalizado(producto):
     '''Producto='trigo','soja','maiz'''
     if producto=='soja':
          df=soja
     elif producto=='trigo':
          df=trigo
     elif producto=='maiz':
          df=maiz
     
     precio_soja_boxplot=px.box(
     df.round(1),
     # x='fecha',
     x='fob_unitario_ton_capit',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob_unitario_ton_capit':'Precio capitalizado',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'fob_unitario_ton_capit',
                    'bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= '', 
                                   # range=[producto['fob_unitario_ton_capit'].min()-100,producto['fob_unitario_ton_capit'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='Fob unitario')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=700,
                                   template = 'none',
                                   title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Capitalizado al último precio disponible. Precio de referencia del Banco Mundial',
                                   showlegend=False)
     
     precio_soja_boxplot.add_vline(x=df.bm.iloc[-1],line_dash="dash",line_color="blue")

     return precio_soja_boxplot

def precio_boxplot(producto):
     '''Producto='trigo','soja','maiz'''
     if producto=='soja':
          df=soja
     elif producto=='trigo':
          df=trigo
     elif producto=='maiz':
          df=maiz
          
     precio_soja_boxplot=px.box(
     df.round(1),
     # x='fecha',
     x='fob_unitario_ton',
     # color='fecha',
     labels={
          "pais_descri": "Destino",
          'fob_unitario_ton': 'Precio',
          'fob':'Fob',
          'pnet':'Kg',
          'fecha':'Fecha',
          'bm': 'Precio de referencia',
          'diferencia_ref':'Spread',
          'empresa':'Exportador'
          },
     hover_data={'pais_descri',
                    'fob_unitario_ton',
                    'bm',
                    'diferencia_ref',
                    'empresa',
                    'fob',
                    'pnet'        
                                   }
     )

     precio_soja_boxplot.update_yaxes(title_text= '', 
                                   # range=[df['fob_unitario_ton'].min()-100,df['fob_unitario_ton'].max()+100]
                                   )
     precio_soja_boxplot.update_xaxes(title_text='Fob unitario')

     precio_soja_boxplot.update_layout(separators=',.', font_family='Georgia', font_size=13,
                                   height=400, width=700,
                                   template = 'none',
                                   title_text=f'FOB por tonelada exportada de {producto} {desde}-{hasta} <br> <sup> Precio de referencia del Banco Mundial',
                                   showlegend=False)

     return precio_soja_boxplot



In [8]:
plot_precio('diferencia_ref','pais_descri','soja')

### Tabla 1
Los valores son a precios capitalizados

In [208]:
def paises_interesantes(producto):
    '''Producto='trigo','soja','maiz'''
    if producto=='soja':
        df=soja
    elif producto=='trigo':
        df=trigo
    elif producto=='maiz':
        df=maiz
        
    contar_paises=df.groupby('pais_descri').size().sort_values(ascending=False)
    paises=pd.DataFrame(contar_paises).reset_index().pais_descri.unique()[:9]
    return paises

paises_interesantes(producto='soja')

array(['Chile', 'Estados unidos', 'China', 'Uruguay', 'Brasil', 'Canada',
       'Paraguay', 'Egipto', 'Taiwan'], dtype=object)

In [209]:
def datos_tabla_paises(pais, producto):
    '''Producto='trigo','soja','maiz'''
    if producto=='soja':
        df=soja
    elif producto=='trigo':
        df=trigo
    elif producto=='maiz':
        df=maiz
    return [len(df[df.pais_descri==pais]),round(df[df.pais_descri==pais].fob_unitario_ton_capit.min(),1), round(df[df.pais_descri==pais].fob_unitario_ton_capit.quantile(0.25),1),round(df[df.pais_descri==pais].fob_unitario_ton_capit.quantile(0.5),1),round(df[df.pais_descri==pais].fob_unitario_ton_capit.quantile(0.75),1),round(df[df.pais_descri==pais].fob_unitario_ton_capit.max(),1)]

# A precios capitalizados

def tabla_1(producto):
    '''Producto='trigo','soja','maiz'''
    if producto=='soja':
        df=soja
    elif producto=='trigo':
        df=trigo
    elif producto=='maiz':
        df=maiz
        
    tabla_1_expo=pd.DataFrame({
        '':['No. of trade records','Lowest price','Lower quartile price','Median price','Upper quartile price','Highest price'],
        'Mundo':[len(df),round(df.fob_unitario_ton_capit.min(),1), round(df.fob_unitario_ton_capit.quantile(0.25),1),round(df.fob_unitario_ton_capit.quantile(0.5),1),round(df.fob_unitario_ton_capit.quantile(0.75),1),round(df.fob_unitario_ton_capit.max(),1)],
        # paises_interesantes[0]:datos_tabla_paises(paises_interesantes[0]),
        **{pais_interesante: datos_tabla_paises(pais_interesante, producto) for pais_interesante in paises_interesantes(producto)}
        })
    tabla_1_expo=tabla_1_expo.transpose().reset_index()
    header=tabla_1_expo.iloc[0]
    tabla_1_expo=tabla_1_expo[1:]
    tabla_1_expo.columns=header
    tabla_1_expo['No. of trade records']=tabla_1_expo['No. of trade records'].astype(int)
    tabla_1_expo.rename(columns={'':'Paises'},inplace=True)
    return tabla_1_expo



### Tabla de las posibles falsificaciones

In [210]:
def tabla_posibles_falsificaciones(producto):
    '''Producto='trigo','soja','maiz'''
    if producto=='soja':
        df=soja
    elif producto=='trigo':
        df=trigo
    elif producto=='maiz':
        df=maiz
    
    posibles_falsificaciones=df[(df.fob_unitario_ton_capit<df.fob_unitario_ton_capit.quantile(.25)) |
                (df.fob_unitario_ton_capit>df.fob_unitario_ton_capit.quantile(.75))]
    # posibles_falsificaciones.drop('index',axis=1,inplace=)
    return posibles_falsificaciones

# Exportar

In [211]:
plot_precio(y='diferencia_ref',color='pais_descri',producto='soja').write_html('../output/diferencia con ref soja pais.html')
plot_precio(y='fob_unitario_ton',color='pais_descri',producto='soja').write_html('../output/fob unitario por ton soja pais.html')
plot_precio(y='diferencia_ref',color='pais_descri',producto='trigo').write_html('../output/diferencia con ref trigo pais.html')
plot_precio(y='diferencia_ref',color='pais_descri',producto='maiz').write_html('../output/diferencia con ref maiz pais.html')
precio_boxplot('soja').write_html('../output/Expo soja boxplot sin capitalizar.html')
precio_boxplot_capitalizado('trigo').write_html('../output/Expo trigo boxplot capitalizado.html')
precio_boxplot_capitalizado('maiz').write_html('../output/Expo maiz boxplot capitalizado.html')
precio_boxplot_capitalizado('soja').write_html('../output/Expo soja boxplot capitalizado.html')

In [212]:
writer = pd.ExcelWriter(f'../output/exportaciones soja trigo maiz.xlsx', engine='xlsxwriter')
tabla_1('soja').to_excel(writer, sheet_name='soja', index=False)
tabla_1('trigo').to_excel(writer, sheet_name='trigo', index=False)
tabla_1('maiz').to_excel(writer, sheet_name='maiz', index=False)
writer.save()

In [ ]:
writer = pd.ExcelWriter(f'../output/exportaciones soja.xlsx', engine='xlsxwriter')
tabla_1('soja').to_excel(writer, sheet_name='tabla_1_expo', index=False)
soja.to_excel(writer, sheet_name='desagregado', index=False)
soja_outliers.head(10).to_excel(writer, sheet_name='outliers', index=False)
tabla_posibles_falsificaciones('soja').to_excel(writer, sheet_name='posibles_falsificaciones')
writer.save()